<a href="https://colab.research.google.com/github/sho20204114/20204114/blob/master/python_ketugou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

①必要・不必要なライブラリをインストール・アンインストールする（ノートを開いたときのみ実行する）。

また、必ずGPUが使用できる状態にしておく

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip list | grep numpy
!pip install -U numba
!pip check
!conda install numpy==1.19.5

numpy                         1.18.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
kapre 0.3.7 requires tensorflow, which is not installed.
xarray-einstats 0.2.2 has requirement numpy>=1.21, but you have numpy 1.18.5.
tables 3.7.0 has requirement numpy>=1.19.0, but you have numpy 1.18.5.
nbclient 0.6.4 has requirement jupyter-client>=6.1.5, but you have jupyter-client 5.3.5.
nbclient 0.6.4 has requirement traitlets>=5.2.2, but you have traitlets 5.1.1.
jaxlib 0.3.7+cuda11.cudnn805 has requirement numpy>=1.19, but you have numpy 1.18.5.
jax 0.3.8 has requirement numpy>=1.19, but you have numpy 1.18.5.
datascience 0.10.6 has requirement folium==0.2.1, but you have folium 0.8.3.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9.
/bin/bash: conda: command not found


In [2]:
!pip uninstall tensorflow -y
!pip install pydub
!pip install natsort
!pip install spleeter-gpu==2.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


②関数を定義する（ノートを開いたときのみ実行する）。

In [3]:
# 全ての警告文を非表示にする(意味不明なエラー文がめちゃくちゃ出てくるので非表示にします。結果に問題はありません。)
import warnings 
warnings.simplefilter('ignore')
 
# ローカルファイルをアップロードすることができる
from google.colab import files

import glob
import os
import re
import shutil
import sys
# リストを自然順アルゴリズムでソートするライブラリ
from natsort import natsorted
# オーディオファイルを音声処理するモジュール
from pydub import AudioSegment 
# 今回の主役。ディープラーニングを利用しある特定のパートの音のみの抽出するモジュール
from spleeter.separator import Separator

# 初期化
def init():
    if os.path.isfile('/content/drive/MyDrive/Colab_Notebooks/tmp_txt.txt') == True:
        os.remove('/content/drive/MyDrive/Colab_Notebooks/tmp_txt.txt')
        print("Deleted the tmp_txt.txt")
    if os.path.isdir('/content/drive/MyDrive/Colab_Notebooks/tmp/') == True:
        shutil.rmtree('/content/drive/MyDrive/Colab_Notebooks/tmp/')
        print("Deleted the tmp folder")

# tmpファイルを削除する
def delete_dir():
    os.remove('/content/drive/MyDrive/Colab_Notebooks/tmp_txt.txt')
    shutil.rmtree('/content/drive/MyDrive/Colab_Notebooks/tmp/')

# 音源を.wavに変換する
def conv_wav(file_name, cnt):
    name = os.path.basename(file_name)
    shutil.copyfile(file_name, '/content/drive/MyDrive/Colab_Notebooks/tmp/{}.{}'.format(name, 'wav'))

    return name
# 入力されたモードから抽出するモードを指定する
def separator_mode(mode):
    print(mode)
    if mode == 2:
        print("mode{}:Vocal & Instruments".format(mode))
        separator = Separator('spleeter:{}stems'.format(mode))
        return separator
    elif mode == 4:
        print("mode{}:Vocal & Bass & Drums".format(mode))
        separator = Separator('spleeter:{}stems'.format(mode))
        return separator
    elif mode == 5:
        print("mode{}:Vocal & Piano & Bass & Drums".format(mode))
        separator = Separator('spleeter:{}stems'.format(mode))
        return separator
    else:
        print("Invalid number：無効なモードが入力されました。次のどれかを指定してください。[2, 4, 5]")
        return 1

③音源から各パートを抽出。また実行する際は以下のことを確認する

・音源がmusicフォルダに入っているか確認する

・spleeter_output内に何も入っていないか（前に作成したファイルが入っている場合上書きされたりして消える場合がある）

・音源は英語表記（日本語を使っているとエラーになる場合がある）

・抽出するモードは適正か

・変換したい音源は一つのフォルダに入れておけば、まとめて処理

In [4]:
def main():
    # 抽出するモードを選択
    # [2->ボーカル&楽器, 4->ボーカル&ベース&ドラム, 5->ボーカル&ピアノ&ベース&ドラム&それ以外]
    mode = 5

    # 抽出したい音源が入っているフォルダを指定
    target_folder_name = '/content/drive/MyDrive/Colab_Notebooks/music/*'
    music_file_name_list = glob.glob(target_folder_name)

    cnt = 1

    # オーディオファイルを.wavに変換し、tmpフォルダに入れる
    for file in natsorted(music_file_name_list):
        print(file)
        os.makedirs('/content/drive/MyDrive/Colab_Notebooks/tmp', exist_ok=True)
        _, ext = os.path.splitext(file)
        if not ext == '.wav':
            bname = conv_wav(file, cnt)
        elif ext == '.wav':
            name = os.path.basename(file)
            shutil.copyfile(file, '/content/drive/MyDrive/Colab_Notebooks/tmp/{}.wav'.format(name))
            bname, _ = os.path.splitext(file)
        with open('/content/drive/MyDrive/Colab_Notebooks/tmp_txt.txt', "a", encoding='utf-8') as f:
            f.write(bname + '\n')
        cnt += 1

    tmp_file = '/content/drive/MyDrive/Colab_Notebooks/tmp/*'
    file = glob.glob(tmp_file)
    # outputフォルダを作成
    os.makedirs('/content/drive/MyDrive/Colab_Notebooks/spleeter_output', exist_ok=True)
    # tmpフォルダ内のデータを一つずつ取り出し処理する
    with open('/content/drive/MyDrive/Colab_Notebooks/tmp_txt.txt') as f:
        title_list = f.read().split("\n")
        #title_list = f.read()
        for data in natsorted(file):
            name = os.path.basename(data)
            separator = separator_mode(mode=mode)
            if separator == 1:
                f.close()
                delete_dir()
                sys.exit()
            # 指定されたモードより、音源からパートを抽出します
            separator.separate_to_file('/content/drive/MyDrive/Colab_Notebooks/tmp/{}'.format(name), '/content/drive/MyDrive/Colab_Notebooks/spleeter_output/', codec="mp3", bitrate="128k")

    delete_dir()

if __name__ == "__main__":
    init()
    main()

/content/drive/MyDrive/Colab_Notebooks/music/gyutto.mp3
/content/drive/MyDrive/Colab_Notebooks/music/koiuta.mp3
/content/drive/MyDrive/Colab_Notebooks/music/love so sweet.mp3
/content/drive/MyDrive/Colab_Notebooks/music/shinderaboy.mp3
/content/drive/MyDrive/Colab_Notebooks/music/yoasobi.mp3
5
mode5:Vocal & Piano & Bass & Drums


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/5stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 5stems archive
INFO:spleeter:5stems model file(s) extracted


INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/5stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for piano_spectrogram
INFO:tensorflow:Apply unet for drums_spectrogram
INFO:tensorflo

INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/vocals.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/piano.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/drums.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/bass.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/other.mp3 written succesfully


5
mode5:Vocal & Piano & Bass & Drums
INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/5stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for piano_spectrogram
INFO:tensorflow:Apply unet

...

INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/piano.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/vocals.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/bass.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/drums.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/other.mp3 written succesfully


5
mode5:Vocal & Piano & Bass & Drums
INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/5stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Apply unet for vocals_spectrogram


Exception ignored in: <generator object Estimator.predict at 0x7faede580ed0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 641, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/usr/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 5588, in get_controller
    yield g
  File "/usr/lib/python3.7/contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 5400, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects
Exception ignored in: <generator object Estimator.predict at 0x7faede708450>
Traceback (most recent call last):
  File "/usr/local/

INFO:tensorflow:Apply unet for piano_spectrogram
INFO:tensorflow:Apply unet for drums_spectrogram
INFO:tensorflow:Apply unet for bass_spectrogram
INFO:tensorflow:Apply unet for other_spectrogram
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from pretrained_models/5stems/model
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/piano.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/vocals.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/bass.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/drums.mp3 written succesfully
INFO:spleeter:File /content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/other.mp3 written succesfully


④結合
pydubをインストールしてあるので、コマンドを打つことで結合できる。

In [13]:
from pydub import AudioSegment

sound1 = AudioSegment.from_file("/content/drive/MyDrive/Colab_Notebooks/spleeter_output/gyutto.mp3/vocals.mp3")
sound2 = AudioSegment.from_file("/content/drive/MyDrive/Colab_Notebooks/spleeter_output/love so sweet.mp3/bass.mp3")
sound3 = AudioSegment.from_file("/content/drive/MyDrive/Colab_Notebooks/spleeter_output/yoasobi.mp3/other.mp3")
sound4 = AudioSegment.from_file("/content/drive/MyDrive/Colab_Notebooks/spleeter_output/koiuta.mp3/piano.mp3")
sound5 = AudioSegment.from_file("/content/drive/MyDrive/Colab_Notebooks/spleeter_output/shinderaboy.mp3/drums.mp3")

output = sound1 + sound2 + sound3 + sound4 + sound5

# save the result
output.export("mixed_sounds.wav", format="wav")

<_io.BufferedRandom name='mixed_sounds.wav'>